In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Import Libraries and Data

In [2]:
from torch._C import ThroughputBenchmark
import os
import pandas as pd
import numpy as np
from PIL import Image
import glob
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, utils, models


import cv2
import plotly.subplots as sp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve
# Training and validation phases are omitted for brevity
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

import sys
from torch.autograd import Variable
import torchvision
from heapq import nsmallest
import time
from operator import itemgetter
import io

# Check if GPU is available and if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

use_cuda = False
if torch.cuda.is_available():
  use_cuda = True
  print("use cuda.")

use cuda.


In [3]:
path = '/content/drive/Shareddrives/CS260cProject/chest_xray'
# train directory
train_folder=path+"/train/"
train_normal_dir=train_folder+"NORMAL/"
train_pneu_dir=train_folder+"PNEUMONIA/"
# test directory
test_folder=path+"/test/"
test_normal_dir=test_folder+"NORMAL/"
test_pneu_dir=test_folder+"PNEUMONIA/"
# validation directory
val_folder=path+"/val/"
val_normal_dir=val_folder+"NORMAL/"
val_pneu_dir=val_folder+"PNEUMONIA/"


In [4]:
# Train Dataset
train_class_names=os.listdir(train_folder)
print("Train class names: %s" % (train_class_names))
# print("\n")

# Validation Dataset
val_class_names=os.listdir(val_folder)
print("Validation class names: %s" % (val_class_names))

# Test Dataset
test_class_names=os.listdir(test_folder)
print("Test class names: %s" % (test_class_names))
# print("\n")

Train class names: ['.DS_Store', 'PNEUMONIA', 'NORMAL']
Validation class names: ['.DS_Store', 'NORMAL', 'PNEUMONIA']
Test class names: ['.DS_Store', 'NORMAL', 'PNEUMONIA']


# Data Extraction

In [5]:
from torch.utils.data import WeightedRandomSampler

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

data_dir = path
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val', 'test']}

class_sample_counts = np.bincount(image_datasets['train'].targets)
class_weights = 1. / torch.tensor(class_sample_counts, dtype=torch.float)
class_weights_normalized = class_weights / class_weights.sum()
samples_weights = class_weights_normalized[image_datasets['train'].targets]
sampler = WeightedRandomSampler(weights=samples_weights, num_samples=len(samples_weights), replacement=True)

dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, sampler=sampler, num_workers=4, pin_memory=True),
    'val': DataLoader(image_datasets['val'], batch_size=32, shuffle=False, num_workers=4, pin_memory=True),
    'test': DataLoader(image_datasets['test'], batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

In [ ]:
class_names

['NORMAL', 'PNEUMONIA']

In [ ]:
class_names

['NORMAL', 'PNEUMONIA']

# EDA

In [ ]:
# Inspect the first 5 samples
# Create a mapping from numerical labels to class names
idx_to_class = {v: k for k, v in image_datasets['train'].class_to_idx.items()}

# Display images from the training dataset
for i in range(5):
    image, label = image_datasets['train'][i]

    # Convert the numerical label back to a class name
    label = idx_to_class[label]
    print(f'Image size: {image.size()}')
    print(f'Label: {label}')

Image size: torch.Size([3, 224, 224])
Label: NORMAL
Image size: torch.Size([3, 224, 224])
Label: NORMAL
Image size: torch.Size([3, 224, 224])
Label: NORMAL
Image size: torch.Size([3, 224, 224])
Label: NORMAL
Image size: torch.Size([3, 224, 224])
Label: NORMAL


In [ ]:
# Get labels directly from the dataset
train_labels = image_datasets['train'].targets
val_labels = image_datasets['val'].targets
test_labels = image_datasets['test'].targets

# Convert numerical labels to their corresponding string labels if necessary
# The class_to_idx attribute is a dictionary that maps class names to numerical labels
idx_to_class = {v: k for k, v in image_datasets['train'].class_to_idx.items()}
train_labels = [idx_to_class[label] for label in train_labels]
val_labels = [idx_to_class[label] for label in val_labels]
test_labels = [idx_to_class[label] for label in test_labels]

fig = make_subplots(rows=1, cols=3, subplot_titles=('Train data', 'Validation data', 'Test data'))

# Add traces
fig.add_trace(go.Histogram(x=train_labels, nbinsx=2, name='Train'), row=1, col=1)
fig.add_trace(go.Histogram(x=val_labels, nbinsx=2, name='Validation'), row=1, col=2)
fig.add_trace(go.Histogram(x=test_labels, nbinsx=2, name='Test'), row=1, col=3)

# Update layout
fig.update_layout(height=400, width=1200, title_text="Diagnosis Distribution")

# Show plot
fig.show()


In [ ]:
from torchvision.transforms import ToPILImage

to_pil = ToPILImage()

# Function to denormalize images
def denormalize(image):
    mean = torch.Tensor([0.485, 0.456, 0.406])
    std = torch.Tensor([0.229, 0.224, 0.225])
    image = image * std[...,None,None] + mean[...,None,None]
    image = image.clamp(0, 1)
    return image

# Convert tensor image to PIL image
def tensor_to_PIL(image):
    image = denormalize(image)
    image = ToPILImage()(image)
    return image

idx_to_class = {v: k for k, v in image_datasets['train'].class_to_idx.items()}
class_labels = [idx_to_class[label] for label in image_datasets['train'].targets]

pneumonia_indices = [i for i, label in enumerate(class_labels) if label == 'Pneumonia'][:4]
normal_indices = [i for i, label in enumerate(class_labels) if label == 'Normal'][:4]

plt.figure(figsize=(20,8))
for i, index in enumerate(pneumonia_indices):
    img, label = image_datasets['train'][index]
    img = tensor_to_PIL(img)
    plt.subplot(2,4,i+1)
    plt.axis('off')
    plt.imshow(img, cmap='gray')
    plt.title('Pneumonia')

for i, index in enumerate(normal_indices):
    img, label = image_datasets['train'][index]
    img = tensor_to_PIL(img)
    plt.subplot(2,4,4+i+1)
    plt.axis('off')
    plt.imshow(img, cmap='gray')
    plt.title('Normal')

plt.show()

<Figure size 2000x800 with 0 Axes>

# Filter Pruning

In [6]:
def replace_layers(model, i, indexes, layers):
    if i in indexes:
        return layers[indexes.index(i)]
    return model[i]

def prune_conv_layer(model, layer_index, filter_index):
    _, conv = list(model.features._modules.items())[layer_index]
    next_conv = None
    offset = 1

    while layer_index + offset < len(model.features._modules.items()):
        res =  list(model.features._modules.items())[layer_index+offset]
        if isinstance(res[1], torch.nn.modules.conv.Conv2d):
            next_name, next_conv = res
            break
        offset = offset + 1

    new_conv = \
        torch.nn.Conv2d(in_channels = conv.in_channels, \
            out_channels = conv.out_channels - 1,
            kernel_size = conv.kernel_size, \
            stride = conv.stride,
            padding = conv.padding,
            dilation = conv.dilation,
            groups = conv.groups,
            bias = (conv.bias is not None))

    old_weights = conv.weight.data.cpu().numpy()
    new_weights = new_conv.weight.data.cpu().numpy()

    new_weights[: filter_index, :, :, :] = old_weights[: filter_index, :, :, :]
    new_weights[filter_index : , :, :, :] = old_weights[filter_index + 1 :, :, :, :]
    new_conv.weight.data = torch.from_numpy(new_weights)
    if use_cuda:
        new_conv.weight.data = new_conv.weight.data.cuda()

    bias_numpy = conv.bias.data.cpu().numpy()

    bias = np.zeros(shape = (bias_numpy.shape[0] - 1), dtype = np.float32)
    bias[:filter_index] = bias_numpy[:filter_index]
    bias[filter_index : ] = bias_numpy[filter_index + 1 :]
    new_conv.bias.data = torch.from_numpy(bias)
    if use_cuda:
        new_conv.bias.data = new_conv.bias.data.cuda()

    if not next_conv is None:
        next_new_conv = \
            torch.nn.Conv2d(in_channels = next_conv.in_channels - 1,\
                out_channels =  next_conv.out_channels, \
                kernel_size = next_conv.kernel_size, \
                stride = next_conv.stride,
                padding = next_conv.padding,
                dilation = next_conv.dilation,
                groups = next_conv.groups,
                bias = (next_conv.bias is not None))

        old_weights = next_conv.weight.data.cpu().numpy()
        new_weights = next_new_conv.weight.data.cpu().numpy()

        new_weights[:, : filter_index, :, :] = old_weights[:, : filter_index, :, :]
        new_weights[:, filter_index : , :, :] = old_weights[:, filter_index + 1 :, :, :]
        next_new_conv.weight.data = torch.from_numpy(new_weights)
        if use_cuda:
            next_new_conv.weight.data = next_new_conv.weight.data.cuda()

        next_new_conv.bias.data = next_conv.bias.data

    if not next_conv is None:
        features = torch.nn.Sequential(
                *(replace_layers(model.features, i, [layer_index, layer_index+offset], \
                    [new_conv, next_new_conv]) for i, _ in enumerate(model.features)))
        del model.features
        del conv

        model.features = features

    else:
        #Pruning the last conv layer. This affects the first linear layer of the classifier.
        model.features = torch.nn.Sequential(
                *(replace_layers(model.features, i, [layer_index], \
                    [new_conv]) for i, _ in enumerate(model.features)))
        layer_index = 0
        old_linear_layer = None
        for _, module in model.classifier._modules.items():
            if isinstance(module, torch.nn.Linear):
                old_linear_layer = module
                break
            layer_index = layer_index  + 1

        if old_linear_layer is None:
            raise BaseException("No linear layer found in classifier")
        params_per_input_channel = old_linear_layer.in_features // conv.out_channels

        new_linear_layer = \
            torch.nn.Linear(old_linear_layer.in_features - params_per_input_channel,
                old_linear_layer.out_features)

        old_weights = old_linear_layer.weight.data.cpu().numpy()
        new_weights = new_linear_layer.weight.data.cpu().numpy()

        new_weights[:, : filter_index * params_per_input_channel] = \
            old_weights[:, : filter_index * params_per_input_channel]
        new_weights[:, filter_index * params_per_input_channel :] = \
            old_weights[:, (filter_index + 1) * params_per_input_channel :]

        new_linear_layer.bias.data = old_linear_layer.bias.data

        new_linear_layer.weight.data = torch.from_numpy(new_weights)
        if use_cuda:
            new_linear_layer.weight.data = new_linear_layer.weight.data.cuda()

        classifier = torch.nn.Sequential(
            *(replace_layers(model.classifier, i, [layer_index], \
                [new_linear_layer]) for i, _ in enumerate(model.classifier)))

        del model.classifier
        del next_conv
        del conv
        model.classifier = classifier

    return model

In [12]:
class FilterPruner:
    def __init__(self, model):
        self.model = model
        self.reset()

    def reset(self):
        self.filter_ranks = {}

    def forward(self, x):
        self.activations = []
        self.gradients = []
        self.grad_index = 0
        self.activation_to_layer = {}

        activation_index = 0
        for layer, (name, module) in enumerate(self.model.features._modules.items()):
            x = module(x)
            if isinstance(module, torch.nn.modules.conv.Conv2d):
                x.register_hook(self.compute_rank)
                self.activations.append(x)
                self.activation_to_layer[activation_index] = layer
                activation_index += 1

        return self.model.classifier(x.view(x.size(0), -1))

    def compute_rank(self, grad):
        activation_index = len(self.activations) - self.grad_index - 1
        activation = self.activations[activation_index]

        taylor = activation * grad
        # Get the average value for every filter,
        # accross all the other dimensions
        taylor = taylor.mean(dim=(0, 2, 3)).data


        if activation_index not in self.filter_ranks:
            self.filter_ranks[activation_index] = \
                torch.FloatTensor(activation.size(1)).zero_()

            if use_cuda:
                self.filter_ranks[activation_index] = self.filter_ranks[activation_index].cuda()

        self.filter_ranks[activation_index] += taylor
        self.grad_index += 1

    def lowest_ranking_filters(self, num):
        data = []
        for i in sorted(self.filter_ranks.keys()):
            for j in range(self.filter_ranks[i].size(0)):
                data.append((self.activation_to_layer[i], j, self.filter_ranks[i][j]))

        return nsmallest(num, data, itemgetter(2))

    def normalize_ranks_per_layer(self):
        for i in self.filter_ranks:
            v = torch.abs(self.filter_ranks[i])
            v = v.cpu()
            v = v / np.sqrt(torch.sum(v * v))
            self.filter_ranks[i] = v.cpu()

    def get_pruning_plan(self, num_filters_to_prune):
        filters_to_prune = self.lowest_ranking_filters(num_filters_to_prune)

        # After each of the k filters are pruned,
        # the filter index of the next filters change since the model is smaller.
        filters_to_prune_per_layer = {}
        for (l, f, _) in filters_to_prune:
            if l not in filters_to_prune_per_layer:
                filters_to_prune_per_layer[l] = []
            filters_to_prune_per_layer[l].append(f)

        for l in filters_to_prune_per_layer:
            filters_to_prune_per_layer[l] = sorted(filters_to_prune_per_layer[l])
            for i in range(len(filters_to_prune_per_layer[l])):
                filters_to_prune_per_layer[l][i] = filters_to_prune_per_layer[l][i] - i

        filters_to_prune = []
        for l in filters_to_prune_per_layer:
            for i in filters_to_prune_per_layer[l]:
                filters_to_prune.append((l, i))

        return filters_to_prune

class PruningFineTuner:
    def __init__(self, model):
        # self.train_data_loader = dataset.loader(train_path)
        # self.test_data_loader = dataset.test_loader(test_path)

        self.model = model
        self.criterion = torch.nn.CrossEntropyLoss()
        self.pruner = FilterPruner(self.model)
        self.model.train()

    def test(self):
        self.model.eval() # Set the model to evaluation mode
        test_correct = 0
        test_total = 0

        test_preds = []
        test_probs = []
        test_labels = []

        with torch.no_grad(): # We don't need gradients for the test phase
            for inputs, labels in dataloaders['test']:
                if use_cuda:
                    inputs = inputs.cuda()
                    labels = labels.cuda()
                # inputs = inputs.to(device)
                # labels = labels.to(device)

                outputs = self.model(inputs)
                _, preds = torch.max(outputs, 1)

                test_total += labels.size(0)
                test_correct += (preds == labels).sum().item()

                # Collect predictions and labels for test set
                test_preds.extend(preds.cpu().numpy())
                test_probs.extend(outputs[:, 1].cpu().numpy())  # Save the probability of the positive class
                test_labels.extend(labels.data.cpu().numpy())

        print('-' * 10)
        print('Accuracy on the test set: %d %%' % (100 * test_correct / test_total))

        # Calculate precision, recall, and F1-score for the test set
        test_preds = np.array(test_preds)
        test_probs = np.array(test_probs) # Convert to numpy array
        test_labels = np.array(test_labels)

        test_precision = precision_score(test_labels, test_preds)
        test_recall = recall_score(test_labels, test_preds)
        test_f1 = f1_score(test_labels, test_preds)
        test_auc = roc_auc_score(test_labels, test_probs) # Here we compute the AUC score using the test labels and predicted probabilities

        print('Test Precision: {:.4f} Recall: {:.4f} F1-score: {:.4f} AUC: {:.4f}'.format(test_precision, test_recall, test_f1, test_auc))

        # Get the best threshold for the precision-recall curve
        precision, recall, thresholds = precision_recall_curve(test_labels, test_probs)

        # Compute F1 score for each threshold
        f1_scores = 2*recall*precision / (recall + precision)

        # Get the threshold that gives the maximum F1 score
        best_threshold = thresholds[np.argmax(f1_scores)]
        print('Best Threshold: ', best_threshold)

        self.model.train()

    def train(self, optimizer = None, epoches=7):
        if optimizer is None:
            optimizer = optim.Adam(self.model.classifier.parameters(), lr=0.001)

        for i in range(epoches):
            # print("Epoch: ", i)
            print('Epoch {}/{}'.format(i+1, epoches))
            print('-' * 10)
            self.train_epoch(optimizer, phase = 'train')
            self.train_epoch(optimizer, phase = "val")

        self.test()
        print("Finished training")

    def train_epoch(self, optimizer = None, rank_filters = False, phase = 'train'):
        # for i, (batch, label) in enumerate(self.train_data_loader):
        # self.train_batch(optimizer, batch, label, rank_filters)

        if phase == 'train':
            self.model.train()  # Set model to training mode
        else:
            self.model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data
        for inputs, labels in dataloaders[phase]:
            if use_cuda:
                inputs = inputs.cuda()
                labels = labels.cuda()

            if (rank_filters):
                self.model.zero_grad()
            else:
                # Zero the parameter gradients
                optimizer.zero_grad()

            # Forward
            # Track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                if (phase == 'train'):
                    # When pruning filters, use the pruner's forward function
                    if rank_filters:
                        outputs = self.pruner.forward(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = self.criterion(outputs, labels)
                        loss.backward()
                    # Backward + optimize only if in training phase
                    else:
                        outputs = self.model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = self.criterion(outputs, labels)
                        loss.backward()
                        optimizer.step()
                # Eval
                else:
                    outputs = self.model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = self.criterion(outputs, labels)

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        # Don't print the results when ranking filters
        if not rank_filters:
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

        # Calculate precision, recall, and F1-score
        preds_cpu = preds.cpu().numpy()
        labels_cpu = labels.data.cpu().numpy()

        precision = precision_score(labels_cpu, preds_cpu)
        recall = recall_score(labels_cpu, preds_cpu)
        f1 = f1_score(labels_cpu, preds_cpu)


    def get_candidates_to_prune(self, num_filters_to_prune):
        self.pruner.reset()
        self.train_epoch(rank_filters = True, phase = "train")
        self.pruner.normalize_ranks_per_layer()
        return self.pruner.get_pruning_plan(num_filters_to_prune)

    def total_num_filters(self):
        filters = 0
        for name, module in self.model.features._modules.items():
            if isinstance(module, torch.nn.modules.conv.Conv2d):
                filters = filters + module.out_channels

        return filters

    def prune(self):
        # Get the accuracy before pruning
        self.test()
        self.model.train()

        #Make sure all the layers are trainable
        for param in self.model.features.parameters():
            param.requires_grad = True

        number_of_filters = self.total_num_filters()
        print("Number of filters: ", number_of_filters)
        num_filters_to_prune_per_iteration = 512
        iterations = int(float(number_of_filters) / num_filters_to_prune_per_iteration)

        iterations = int(iterations * 2.0 / 3)

        print("Number of pruning iterations to reduce 67% filters: ", iterations)

        for i in range(iterations):
            print("\nPruning iteration: ", i+1)
            print("Ranking filters... ")
            prune_targets = self.get_candidates_to_prune(num_filters_to_prune_per_iteration)
            layers_pruned = {}
            for layer_index, filter_index in prune_targets:
                if layer_index not in layers_pruned:
                    layers_pruned[layer_index] = 0
                layers_pruned[layer_index] = layers_pruned[layer_index] + 1

            print("Layers that will be pruned", layers_pruned)
            print("Pruning filters... ")
            model = self.model.cpu()
            for layer_index, filter_index in prune_targets:
                model = prune_conv_layer(model, layer_index, filter_index)

            self.model = model
            if use_cuda:
                self.model = self.model.cuda()

            # message = str(100*float(self.total_num_filters()) / number_of_filters) + "%"
            print("Number of filters after pruning: ", self.total_num_filters())
            self.test()
            print("Fine tuning to recover from pruning iteration...")
            optimizer = optim.Adam(self.model.parameters(), lr=0.001)
            self.train(optimizer, epoches=3)


        # print("Finished. Fine tune the model a bit more...")
        # self.train(optimizer, epoches=3)
        torch.save(model.state_dict(), "model_pruned")

# Model Training

# VGG16

In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve

# Check if GPU is available and if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained VGG16, and freeze the weights
vgg16 = models.vgg16(pretrained=True)
for param in vgg16.features.parameters():
    param.requires_grad = False

# Modify the classifier layer to match the number of classes in the dataset
num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1]  # Remove last layer
features.extend([nn.Linear(num_features, len(class_names))])  # Add our layer with 2 outputs
vgg16.classifier = nn.Sequential(*features)  # Replace the model classifier

# Move the model to the device
# vgg16 = vgg16.to(device)

# Define the loss and the optimizer
criterion = nn.CrossEntropyLoss()

# Only parameters of the final layer are being optimized
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=0.001)

# Number of epochs
epochs = 7

# Train the model
model = vgg16
if use_cuda:
    model = model.cuda()
fine_tuner = PruningFineTuner(model)
fine_tuner.train(epoches=epochs)
torch.save(model, "model")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 194MB/s]


Epoch 1/7
----------
train Loss: 0.5130 Acc: 0.8248
val Loss: 1.2163 Acc: 0.6250
Epoch 2/7
----------
train Loss: 0.3899 Acc: 0.8679
val Loss: 0.8042 Acc: 0.5625
Epoch 3/7
----------
train Loss: 0.3582 Acc: 0.8731
val Loss: 0.7016 Acc: 0.7500
Epoch 4/7
----------
train Loss: 0.3626 Acc: 0.8760
val Loss: 0.2206 Acc: 0.9375
Epoch 5/7
----------
train Loss: 0.3156 Acc: 0.8953
val Loss: 0.1624 Acc: 0.9375
Epoch 6/7
----------
train Loss: 0.3191 Acc: 0.8944
val Loss: 0.3325 Acc: 0.8750
Epoch 7/7
----------
train Loss: 0.3241 Acc: 0.8834
val Loss: 0.2760 Acc: 0.9375
----------
Accuracy on the test set: 88 %
Test Precision: 0.8514 Recall: 0.9846 F1-score: 0.9132 AUC: 0.9607
Best Threshold:  0.7226939
Finished training


In [9]:
# Calculate the number of parameters after training
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

total_params, trainable_params = count_parameters(model)
print(f"Total parameters before pruning: {total_params}")
print(f"Trainable parameters before pruning: {trainable_params}")

def model_size_in_MB(model):
    buffer = io.BytesIO()
    torch.save(model.state_dict(), buffer)
    return buffer.tell() / (1024 * 1024)  # Convert bytes to MB

model_size = model_size_in_MB(model)
print(f"Model size before pruning: {model_size} MB")

# Measure the inference time
# input, label = next(iter(dataloaders["test"]))
# if use_cuda:
#     input = input.cuda()

t0 = time.time()
fine_tuner.test()
print("The inference took", time.time() - t0, "sec")

Total parameters before pruning: 134268738
Trainable parameters before pruning: 119554050
Model size before pruning: 512.2047166824341 MB
----------
Accuracy on the test set: 88 %
Test Precision: 0.8514 Recall: 0.9846 F1-score: 0.9132 AUC: 0.9607
Best Threshold:  0.7226939
The inference took 3.9122323989868164 sec


In [13]:
# Prune the model
model = torch.load("model", map_location=lambda storage, loc: storage)
if use_cuda:
    model = model.cuda()
fine_tuner = PruningFineTuner(model)
fine_tuner.prune()

----------
Accuracy on the test set: 88 %
Test Precision: 0.8514 Recall: 0.9846 F1-score: 0.9132 AUC: 0.9607
Best Threshold:  0.7226939
Number of filters:  4224
Number of pruning iterations to reduce 67% filters:  5

Pruning iteration:  1
Ranking filters... 
Layers that will be pruned {17: 59, 21: 64, 28: 113, 19: 65, 24: 80, 10: 20, 14: 11, 12: 10, 26: 70, 7: 6, 2: 4, 0: 4, 5: 6}
Pruning filters... 
Number of filters after pruning:  3712
----------
Accuracy on the test set: 89 %
Test Precision: 0.8854 Recall: 0.9513 F1-score: 0.9172 AUC: 0.9588
Best Threshold:  0.3281502
Fine tuning to recover from pruning iteration...
Epoch 1/3
----------
train Loss: 0.9538 Acc: 0.6089
val Loss: 0.5149 Acc: 0.7500
Epoch 2/3
----------
train Loss: 0.5203 Acc: 0.7690
val Loss: 0.5392 Acc: 0.6875
Epoch 3/3
----------
train Loss: 0.4676 Acc: 0.8090
val Loss: 0.8477 Acc: 0.6250
----------
Accuracy on the test set: 73 %
Test Precision: 0.7121 Recall: 0.9641 F1-score: 0.8192 AUC: 0.8894
Best Threshold:  0.4

In [14]:
# Calculate the number of parameters and model size after pruning
total_params, trainable_params = count_parameters(model)
print(f"Total parameters after pruning: {total_params}")
print(f"Trainable parameters after pruning: {trainable_params}")

model_size = model_size_in_MB(model)
print(f"Model size after pruning: {model_size} MB")

# Measure the inference time
# input, label = next(iter(dataloaders["test"]))
# if use_cuda:
#     input = input.cuda()

t0 = time.time()
fine_tuner.test()
print("The inference took", time.time() - t0)

Total parameters after pruning: 98600067
Trainable parameters after pruning: 98600067
Model size after pruning: 376.13910388946533 MB
----------
Accuracy on the test set: 84 %
Test Precision: 0.8156 Recall: 0.9641 F1-score: 0.8837 AUC: 0.9164
Best Threshold:  0.25637606
The inference took 4.13808274269104


## VGG19

In [15]:
# Load pre-trained VGG16, and freeze the weights
vgg19 = models.vgg19(pretrained=True)
for param in vgg19.features.parameters():
  param.requires_grad = False

# Modify the classifier layer to match the number of classes in the dataset
num_features = vgg19.classifier[6].in_features
features = list(vgg19.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 2 outputs
vgg19.classifier = nn.Sequential(*features) # Replace the model classifier

# Define the loss and the optimizer
criterion = nn.CrossEntropyLoss()

# Only parameters of the final layer are being optimized
optimizer = optim.Adam(vgg19.classifier.parameters(), lr=0.001)

# Number of epochs
epochs = 7

# Train the model
model = vgg19
if use_cuda:
    model = model.cuda()
fine_tuner = PruningFineTuner(model)
fine_tuner.train(epoches=epochs)
torch.save(model, "model")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:02<00:00, 215MB/s]


Epoch 1/7
----------
train Loss: 0.5103 Acc: 0.8152
val Loss: 0.3220 Acc: 0.8750
Epoch 2/7
----------
train Loss: 0.3691 Acc: 0.8689
val Loss: 1.0814 Acc: 0.6875
Epoch 3/7
----------
train Loss: 0.3743 Acc: 0.8645
val Loss: 0.3261 Acc: 0.8750
Epoch 4/7
----------
train Loss: 0.3398 Acc: 0.8786
val Loss: 0.1964 Acc: 0.8750
Epoch 5/7
----------
train Loss: 0.3229 Acc: 0.8821
val Loss: 0.2729 Acc: 0.8750
Epoch 6/7
----------
train Loss: 0.3256 Acc: 0.8924
val Loss: 0.6272 Acc: 0.8750
Epoch 7/7
----------
train Loss: 0.3502 Acc: 0.8806
val Loss: 0.3589 Acc: 0.8750
----------
Accuracy on the test set: 86 %
Test Precision: 0.8337 Recall: 0.9769 F1-score: 0.8996 AUC: 0.9562
Best Threshold:  0.8324729
Finished training


In [16]:
# Calculate the number of parameters after training
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

total_params, trainable_params = count_parameters(model)
print(f"Total parameters before pruning: {total_params}")
print(f"Trainable parameters before pruning: {trainable_params}")

def model_size_in_MB(model):
    buffer = io.BytesIO()
    torch.save(model.state_dict(), buffer)
    return buffer.tell() / (1024 * 1024)  # Convert bytes to MB

model_size = model_size_in_MB(model)
print(f"Model size before pruning: {model_size} MB")

# Measure the inference time
# input, label = next(iter(dataloaders["test"]))
# if use_cuda:
#     input = input.cuda()

t0 = time.time()
# output = model(input)
fine_tuner.test()
print("The inference took", time.time() - t0, "sec")

Total parameters before pruning: 139578434
Trainable parameters before pruning: 119554050
Model size before pruning: 532.4615964889526 MB
----------
Accuracy on the test set: 86 %
Test Precision: 0.8337 Recall: 0.9769 F1-score: 0.8996 AUC: 0.9562
Best Threshold:  0.8324729
The inference took 4.070988893508911 sec


In [17]:
# Prune the model
model = torch.load("model", map_location=lambda storage, loc: storage)
if use_cuda:
    model = model.cuda()
fine_tuner = PruningFineTuner(model)
fine_tuner.prune()

----------
Accuracy on the test set: 86 %
Test Precision: 0.8337 Recall: 0.9769 F1-score: 0.8996 AUC: 0.9562
Best Threshold:  0.8324729
Number of filters:  5504
Number of pruning iterations to reduce 67% filters:  6

Pruning iteration:  1
Ranking filters... 
Layers that will be pruned {2: 3, 34: 88, 32: 78, 28: 65, 25: 60, 19: 42, 10: 11, 30: 54, 23: 40, 21: 37, 12: 10, 0: 3, 16: 6, 14: 12, 5: 1, 7: 2}
Pruning filters... 
Number of filters after pruning:  4992
----------
Accuracy on the test set: 84 %
Test Precision: 0.8246 Recall: 0.9641 F1-score: 0.8889 AUC: 0.9347
Best Threshold:  0.51193506
Fine tuning to recover from pruning iteration...
Epoch 1/3
----------
train Loss: 21.2640 Acc: 0.5606
val Loss: 0.6938 Acc: 0.5000
Epoch 2/3
----------
train Loss: 0.8185 Acc: 0.5849
val Loss: 0.5867 Acc: 0.7500
Epoch 3/3
----------
train Loss: 0.7427 Acc: 0.5652
val Loss: 0.6613 Acc: 0.6250
----------
Accuracy on the test set: 53 %
Test Precision: 0.9904 Recall: 0.2641 F1-score: 0.4170 AUC: 0.9

<ipython-input-12-ec0929f224ef>:141: RuntimeWarning: invalid value encountered in true_divide
  f1_scores = 2*recall*precision / (recall + precision)


In [18]:
# Calculate the number of parameters and model size after pruning
total_params, trainable_params = count_parameters(model)
print(f"Total parameters after pruning: {total_params}")
print(f"Trainable parameters after pruning: {trainable_params}")

model_size = model_size_in_MB(model)
print(f"Model size after pruning: {model_size} MB")

# Measure the inference time
t0 = time.time()
output = fine_tuner.test()
print("The inference took", time.time() - t0)

Total parameters after pruning: 107140297
Trainable parameters after pruning: 107140297
Model size after pruning: 408.7192258834839 MB
----------
Accuracy on the test set: 83 %
Test Precision: 0.9152 Recall: 0.8026 F1-score: 0.8552 AUC: 0.9137
Best Threshold:  1.5683963
The inference took 3.997321367263794


<ipython-input-12-ec0929f224ef>:141: RuntimeWarning: invalid value encountered in true_divide
  f1_scores = 2*recall*precision / (recall + precision)


## ResNet-50

### Transfer Learning

In [ ]:
resnet50 = models.resnet50(pretrained=True)
for param in resnet50.parameters():
    param.requires_grad = False

# Modify the classifier layer to match the number of classes in the dataset
num_features = resnet50.fc.in_features
resnet50.fc = nn.Sequential(
    nn.Linear(num_features, 512),   # first linear layer
    nn.ReLU(),

    nn.Linear(512, 256),   # first linear layer
    nn.ReLU(),

    nn.Linear(256, 128),        # second linear layer
    nn.ReLU(),

    nn.Linear(128, 2)           # third linear layer, output size = 2
)

# Move the model to the device
resnet50 = resnet50.to(device)

# Define the loss and the optimizer
criterion = nn.CrossEntropyLoss()

# Only parameters of the final layer are being optimized
optimizer = optim.SGD(resnet50.fc.parameters(), lr=0.001, momentum=0.3)

# Number of epochs
epochs = 12

# Training loop
for epoch in range(epochs):
    print('Epoch {}/{}'.format(epoch+1, epochs))
    print('-' * 10)

    for phase in ['train', 'val']:
        if phase == 'train':
            resnet50.train()  # Set model to training mode
        else:
            resnet50.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward
            # Track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                outputs = resnet50(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))



Epoch 1/12
----------
train Loss: 0.6840 Acc: 0.5991
val Loss: 0.6642 Acc: 0.7500
Epoch 2/12
----------
train Loss: 0.6290 Acc: 0.7659
val Loss: 0.5521 Acc: 0.6875
Epoch 3/12
----------
train Loss: 0.4849 Acc: 0.8002
val Loss: 0.6036 Acc: 0.7500
Epoch 4/12
----------
train Loss: 0.4344 Acc: 0.8016
val Loss: 0.5823 Acc: 0.6250
Epoch 5/12
----------
train Loss: 0.4143 Acc: 0.8160
val Loss: 0.7473 Acc: 0.7500
Epoch 6/12
----------
train Loss: 0.3985 Acc: 0.8265
val Loss: 0.6865 Acc: 0.7500
Epoch 7/12
----------
train Loss: 0.4012 Acc: 0.8225
val Loss: 0.6493 Acc: 0.7500
Epoch 8/12
----------
train Loss: 0.3920 Acc: 0.8269
val Loss: 0.7215 Acc: 0.7500
Epoch 9/12
----------
train Loss: 0.4010 Acc: 0.8196
val Loss: 0.6420 Acc: 0.7500
Epoch 10/12
----------
train Loss: 0.3888 Acc: 0.8288
val Loss: 0.7677 Acc: 0.7500
Epoch 11/12
----------
train Loss: 0.3992 Acc: 0.8257
val Loss: 0.7682 Acc: 0.6875
Epoch 12/12
----------
train Loss: 0.3912 Acc: 0.8292
val Loss: 0.5785 Acc: 0.7500
----------
Ac

NameError: ignored

In [ ]:

# Training and validation phases are omitted for brevity

resnet50.eval() # Set the model to evaluation mode
test_correct = 0
test_total = 0

test_preds = []
test_probs = []
test_labels = []

with torch.no_grad(): # We don't need gradients for the test phase
    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = resnet50(inputs)
        _, preds = torch.max(outputs, 1)

        test_total += labels.size(0)
        test_correct += (preds == labels).sum().item()

        # Collect predictions and labels for test set
        test_preds.extend(preds.cpu().numpy())
        test_probs.extend(torch.nn.functional.softmax(outputs, dim=1)[:, 1].cpu().numpy())  # Save the probability of the positive class
        test_labels.extend(labels.data.cpu().numpy())

print('-' * 10)
print('Accuracy on the test set: %d %%' % (100 * test_correct / test_total))

# Calculate precision, recall, and F1-score for the test set
test_preds = np.array(test_preds)
test_probs = np.array(test_probs) # Convert to numpy array
test_labels = np.array(test_labels)

test_precision = precision_score(test_labels, test_preds)
test_recall = recall_score(test_labels, test_preds)
test_f1 = f1_score(test_labels, test_preds)
test_auc = roc_auc_score(test_labels, test_probs) # Here we compute the AUC score using the test labels and predicted probabilities

print('Test Precision: {:.4f} Recall: {:.4f} F1-score: {:.4f} AUC: {:.4f}'.format(test_precision, test_recall, test_f1, test_auc))

# Get the best threshold for the precision-recall curve
precision, recall, thresholds = precision_recall_curve(test_labels, test_probs)

# Compute F1 score for each threshold
f1_scores = 2*recall*precision / (recall + precision)

# Get the threshold that gives the maximum F1 score
best_threshold = thresholds[np.argmax(f1_scores)]
print('Best Threshold: ', best_threshold)

----------
Accuracy on the test set: 85 %
Test Precision: 0.8889 Recall: 0.8821 F1-score: 0.8855 AUC: 0.9378
Best Threshold:  0.38178363


### Fine-tuning

In [ ]:
resnet50 = models.resnet50(pretrained=True)

# Unfreeze the layers you want to fine-tune
for param in resnet50.layer3.parameters():
    param.requires_grad = True
for param in resnet50.layer4.parameters():
    param.requires_grad = True

# Modify the classifier layer to match the number of classes in the dataset
num_features = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_features, len(class_names))

# Move the model to the device
resnet50 = resnet50.to(device)

# Define the loss and the optimizer
criterion = nn.CrossEntropyLoss()

# Specify the parameters to optimize and their learning rate
optimizer = optim.SGD([
    {'params': resnet50.layer3.parameters(), 'lr': 0.001},
    {'params': resnet50.layer4.parameters(), 'lr': 0.001},
    {'params': resnet50.fc.parameters(), 'lr': 0.01}
], momentum=0.9)

# Number of epochs
epochs = 10

# Training loop
for epoch in range(epochs):
    print('Epoch {}/{}'.format(epoch+1, epochs))
    print('-' * 10)

    for phase in ['train', 'val']:
        if phase == 'train':
            resnet50.train()  # Set model to training mode
        else:
            resnet50.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward
            # Track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                outputs = resnet50(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

# Training and validation phases are omitted for brevity

resnet50.eval() # Set the model to evaluation mode
test_correct = 0
test_total = 0

test_preds = []
test_probs = []
test_labels = []

with torch.no_grad(): # We don't need gradients for the test phase
    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = resnet50(inputs)
        _, preds = torch.max(outputs, 1)

        test_total += labels.size(0)
        test_correct += (preds == labels).sum().item()

        # Collect predictions and labels for test set
        test_preds.extend(preds.cpu().numpy())
        test_probs.extend(torch.nn.functional.softmax(outputs, dim=1)[:, 1].cpu().numpy())  # Save the probability of the positive class
        test_labels.extend(labels.data.cpu().numpy())

print('-' * 10)
print('Accuracy on the test set: %d %%' % (100 * test_correct / test_total))

# Calculate precision, recall, and F1-score for the test set
test_preds = np.array(test_preds)
test_probs = np.array(test_probs) # Convert to numpy array
test_labels = np.array(test_labels)

test_precision = precision_score(test_labels, test_preds)
test_recall = recall_score(test_labels, test_preds)
test_f1 = f1_score(test_labels, test_preds)
test_auc = roc_auc_score(test_labels, test_probs) # Here we compute the AUC score using the test labels and predicted probabilities

print('Test Precision: {:.4f} Recall: {:.4f} F1-score: {:.4f} AUC: {:.4f}'.format(test_precision, test_recall, test_f1, test_auc))

# Get the best threshold for the precision-recall curve
precision, recall, thresholds = precision_recall_curve(test_labels, test_probs)

# Compute F1 score for each threshold
f1_scores = 2*recall*precision / (recall + precision)

# Get the threshold that gives the maximum F1 score
best_threshold = thresholds[np.argmax(f1_scores)]
print('Best Threshold: ', best_threshold)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.



Epoch 1/10
----------
train Loss: 1.0052 Acc: 0.7653
val Loss: 0.5696 Acc: 0.6875
Epoch 2/10
----------
train Loss: 0.4107 Acc: 0.8388
val Loss: 0.6467 Acc: 0.6875
Epoch 3/10
----------
train Loss: 0.3221 Acc: 0.8735
val Loss: 0.4232 Acc: 0.8125
Epoch 4/10
----------
train Loss: 0.2454 Acc: 0.9024
val Loss: 0.2026 Acc: 0.9375
Epoch 5/10
----------
train Loss: 0.2210 Acc: 0.9187
val Loss: 0.1903 Acc: 0.9375
Epoch 6/10
----------
train Loss: 0.1996 Acc: 0.9260
val Loss: 0.2661 Acc: 0.9375
Epoch 7/10
----------
train Loss: 0.1911 Acc: 0.9306
val Loss: 0.3723 Acc: 0.8125
Epoch 8/10
----------
train Loss: 0.1773 Acc: 0.9337
val Loss: 0.2132 Acc: 0.9375
Epoch 9/10
----------
train Loss: 0.1619 Acc: 0.9377
val Loss: 0.2884 Acc: 0.9375
Epoch 10/10
----------
train Loss: 0.1575 Acc: 0.9442
val Loss: 0.0705 Acc: 1.0000
----------
Accuracy on the test set: 94 %
Test Precision: 0.9212 Recall: 0.9897 F1-score: 0.9543 AUC: 0.9893
Best Threshold:  0.8006818
